# Dogs vs Cats - features

[Kaggle](https://www.kaggle.com/c/dogs-vs-cats)

1 = dog

0 = cat

In [71]:
import cv2
from matplotlib import pyplot as plt
import sklearn
import numpy as np
import pickle as pk

plt.style.use('ggplot')
%matplotlib inline

### Load training dataset

In [96]:
train_folder = 'data/train/'

In [47]:
imgs_paths = [train_folder + filepath for filepath in listdir(train_folder)]

In [48]:
# select a subset
imgs_paths = imgs_paths[:100]

In [113]:
from os import listdir

def load_images(imgs_paths, gray=False):
    for path in imgs_paths:
        img = cv2.imread(path)
        
        if gray:
            yield cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        else:
            yield cv2.imread(path)

In [77]:
labels = [1 if "dog" in path else 0 for path in imgs_paths]

In [78]:
print('Nr dogs:', labels.count(1))

Nr dogs: 52


In [79]:
print('Nr cats:', labels.count(0))

Nr cats: 48


### Features Extraction

In [80]:
# SIFT features detector and extractor
sift = cv2.xfeatures2d.SIFT_create()

In [81]:
# SURF features detector and extractor
surf = cv2.xfeatures2d.SURF_create()

In [82]:
# FAST features detector
fast = cv2.FastFeatureDetector_create()

In [83]:
# BRISK descriptors extractor
br = cv2.BRISK_create()

In [173]:
# FLANN matcher
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

flann = cv2.FlannBasedMatcher(index_params,search_params)

In [131]:
def get_descriptors(imgs, detector, extractor=None):
    for img in imgs:
        if extractor == None:
            yield detector(img, None)
        else:
            kp = detector(img, None)
            yield extractor(img, kp)

In [147]:
imgs = load_images(imgs_paths, gray=True)
imgs_sift_des = get_descriptors(imgs, sift.detectAndCompute)

In [148]:
imgs = load_images(imgs_paths, gray=True)
imgs_surf_des = get_descriptors(imgs, surf.detectAndCompute)

In [149]:
imgs = load_images(imgs_paths, gray=True)
imgs_fast_des = get_descriptors(imgs, detector=fast.detect, extractor=br.compute)

### Output tools

In [97]:
test_folder = 'data/test1/'

In [144]:
imgs_paths_test = [test_folder + filepath for filepath in listdir(test_folder)]

### Classifiers

In [150]:
from sklearn.metrics import accuracy_score, confusion_matrix
# function(y_true, y_pred)

In [151]:
from sklearn.cross_validation import KFold
# kf = KFold(nr_samples, n_folds=2)
# returns indexes

In [155]:
from sklearn.cross_validation import ShuffleSplit
# for train, test in ShuffleSplit(nr_samples, n_iter=1, test_size=.3, random_state=0)

#### Nearest Neighbors

In [182]:
from queue import PriorityQueue

def knn(train_paths, test_paths, descriptor, extractor=None, k=5):

    # array to store predictions
    pred_labels = []
    
    # labels
    labels_train = [1 if "dog" in path else 0 for path in train_paths]
    
    # get images
    imgs_train = load_images(train_paths, gray=True)
    imgs_test = load_images(test_paths, gray=True)
    
    # generate test descriptors
    imgs_test_des = get_descriptors(imgs_test, descriptor, extractor)
    
    for test_sample in imgs_test_des:
        
        # queue to store votes
        pq = PriorityQueue()
        
        # generate train descriptors
        imgs_train_des = get_descriptors(imgs_test, descriptor, extractor)
        
        for train_sample, train_label in zip(imgs_train_des, labels_train):
            # match descriptors
            matches = flann.knnMatch(test_sample, train_sample, k=2)
            #matchesMasks = [[0,0] for _ in range(len(matches))]
            
            nr_matches = 0

            # ratio test as per Lowe's paper
            for i in range(len(matches)):
                for j, (m, n) in enumerate(matches[i]):
                    if m.distance < 0.7 * n.distance:
                        #matchesMasks[i][j] = [1,0]
                        nr_matches += 1
            
            pq.put((-nr_matches, train_labels))

        # count votes
        dog_votes = 0
        for i in range(k):
            dog_votes += pq.get()[1]
        
        if dog_votes > k / 2:
            pred_labels.append(1)
        else:
            pred_labels.append(0)
    
    return pred_labels

In [183]:
imgs_paths_np = np.array(imgs_paths)

for train, test in ShuffleSplit(100, n_iter=2, test_size=.3, random_state=0):
    y_pred = knn(imgs_paths_np[train], imgs_paths_np[test], sift.detectAndCompute)
    
    y_true = [1 if "dog" in path else 0 for path in imgs_paths_np[test]]
    
    score = accuracy_score(y_true, y_pred)
    conf_mat = confusion_matrix(y_true, y_pred)
    
    print('Accuracy:', score)
    print('Confusion matrix:', conf_mat, sep='\n')

TypeError: float() argument must be a string or a number, not 'cv2.KeyPoint'